*Copyright: 
Prof. Dr. Christian Wartena, http://textmining.wp.hs-hannover.de/; Frieda Josi*

## Extraktion von Wortpaaren

Schlagworte können auch aus kurzen Phrasen (Wortpaaren) bestehen. Deshalb werden zusätzlich zu der Termextraktion auch Wortpaare exrahiert.

Extraktion von Wortpaaren erfolgt aus den Beschriftungen und Referenzstellen der 397 Abbildungen aus Artikel, die aus Open Access Journals (NOA Projekt http://noa.wp.hs-hannover.de) stammen. Die Artikel haben meist mehrere Abbildungen, die aber nicht alle mit aufgenommen wurden. Es sind 397 einzelne txt-Dateien, angezeigt werden die Datensätze 150 bis 170. 

Kookkurenzen vom Wörter im gleichen Satz.
Lemmata statt Wörter.

In [1]:
import glob
import nltk
import codecs
import re
import treetaggerwrapper
from nltk.corpus import stopwords

swlist = stopwords.words('english')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
filelist = glob.glob("doi-cap-con/*.txt")
nr_of_words = 0
fdist = nltk.FreqDist() #Worthäufigkeiten Anzahl
fdist2 = nltk.FreqDist() #Wortpaarhäufigkeiten Frequenz von Wortpaaren

for datei in filelist:  
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')#Texte aufteilen, Jeder Satz ist eine Liste von Tokens
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]#Daraus Wortpaare und Wörter extrahieren
    words = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        words = [lemma for (word,pos,lemma) in tags2 if pos[0]]#Jede Wortart+ Lemma
        word_pairs = [] #Wortpaare
        for i in range(len(tags2)-1) :#Bereich Liste von Zahlen 0 bis Länge von "tags2" -1. Das nächste letzte Wort soll immer angeschaut werden.
            t1 = tags2[i]# Wort i + i+1, 
            t2 = tags2[i+1]
            if t1.pos[0] and t2.pos[0]:
                l1 = t1.lemma 
                l2 = t2.lemma
                if l1 not in swlist and l2 not in swlist: #neu dazu geschrieben, Stopwörter rausnehmen
                    word_pairs.append((l1,l2)) #10 häufigsten Wortpaare. Nur Häufigkeit, eigentlich ist die Kombination die häufig zu erwartet ist und auch eintrifft.
                
        nr_of_words += len(words)
        fdist.update(words)
        fdist2.update(word_pairs)
        
    
print(fdist2.most_common(3))

[((')', '.'), 1746), (('@card@', ')'), 1608), (('/math', '>'), 1167)]


## Eingrenzung auf ein Muster

Wortartmuster für Kookkurrenzen (Wortartmuster für Nominalphrasen) 

In [2]:
import glob
import nltk
import codecs
import re
import treetaggerwrapper
import pprint
import math
from nltk.corpus import stopwords

swlist = stopwords.words('english')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
filelist = glob.glob("doi-cap-con/*.txt")
nr_of_words = 0
fdist = nltk.FreqDist() #Worthäufigkeiten
fdist2 = nltk.FreqDist() #Wortpaarhäufigkeiten


npmuster = [('NN','NN'), #Nomen
            ('NP','NP'), #Eigennamen
            ('NN','NP'), 
            ('NP','NN'),
            ('JJ','NP'), # Adjektive
            ('JJ','NN')
           ]
    

for datei in filelist:  
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()

    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    words = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        words = [lemma for (word,pos,lemma) in tags2 if pos[0]]
        word_pairs = []
        for i in range(len(tags2)-1) :
            t1 = tags2[i]
            t2 = tags2[i+1]
            if (t1.pos,t2.pos) in npmuster:
                l1 = t1.lemma
                l2 = t2.lemma
                if l1 not in swlist and l2 not in swlist  and len(l1) >2 and len(l2) >2: #Englische Stopwörter
                    word_pairs.append((l1,l2))
        
        nr_of_words += len(words)
        fdist.update(words)
        fdist2.update(word_pairs)
print("Anzahl Wortvorkommen (tokens): ", nr_of_words)
pprint.pprint(fdist2.most_common(100))     


Anzahl Wortvorkommen (tokens):  235570
[(('time', 'series'), 69),
 (('wind', 'speed'), 69),
 (('standard', 'deviation'), 55),
 (('right', 'panel'), 53),
 (('radial', 'velocity'), 51),
 (('accumulation', 'rate'), 36),
 (('study', 'area'), 32),
 (('red', 'line'), 27),
 (('spindle', 'cell'), 26),
 (('water', 'level'), 26),
 (('blue', 'line'), 25),
 (('reference', 'blade'), 24),
 (('CTH', 'scheme'), 22),
 (('camera', 'motion'), 21),
 (('total', 'column'), 21),
 (('velocity', 'spectra'), 21),
 (('relative', 'humidity'), 20),
 (('mass', 'load'), 20),
 (('peritoneal', 'cavity'), 20),
 (('boundary', 'layer'), 20),
 (('ICEFLUX', 'scheme'), 20),
 (('diurnal', 'cycle'), 19),
 (('dental', 'pulp'), 18),
 (('transduction', 'efficiency'), 18),
 (('flash', 'rate'), 18),
 (('bone', 'marrow'), 18),
 (('tip', 'deflection'), 18),
 (('BARC', 'building'), 18),
 (('time', 'period'), 18),
 (('particulate', 'matter'), 17),
 (('lightning', 'activity'), 17),
 (('search', 'region'), 17),
 (('black', 'line'), 17),

## Extraktion von Nominalphrasen aus einzelne Datensätze

Nominalphrasen nach Muster für Extraktion verwenden.
Tokenisierung ändern: Häufigkeit und tf.idf Berechnung (Nominalphrase als einziges Token)


In [3]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper

df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
NPList = fdist2.keys()

def candidates(taglist):#Liste mit Tags, als Ausgabe als Liste
    cand = []
    skip = False
    for i in range(len(taglist)-1) :
        if skip:
            skip = False
            continue
        skip = False
        l1 = taglist[i].lemma # erstes Wort
        l2 = taglist[i+1].lemma# zweites Wort
        if len(l1) >2 and len(l2) >2:   #Die einzelnen Wörter der NP sollen eine Länge von über 2 Buchstaben haben
            if (l1,l2) in NPList:
                skip = True
                cand.append((l1,l2))
    return cand


def substantive_zaehlen(datei):
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = []   
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text:
                nouns_in_text.append(substantiv)
                
    for n in nouns_in_text:
        df_n = df.get(n,0) #Der Wert von n falls n vorhanden ist, sonst 0
        df[n] = df_n + 1

for f in glob.glob("doi-cap-con/*.txt"):
    substantive_zaehlen(f)

Nominalphrasen für einen Datensatz extrahieren:

In [4]:
def extract_kw(datei,nr_of_docs):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True)
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        nouns.extend(nouns_from_sent)

    fdist = nltk.FreqDist(nouns)  
    
        
    for word in fdist:
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word]))
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf
            
    return fdist.most_common(8)
    

filelist = glob.glob("doi-cap-con/*.txt")
nr_of_docs = len(filelist)
for f in filelist[150:170]:
    keywords = extract_kw(f,nr_of_docs)
    pprint.pprint(f)
    pprint.pprint(keywords)
    pprint.pprint("------------------------------")

'doi-cap-con/10.1155:2016:1875357.txt'
[(('time', 'slot'), 0.9106192449462366),
 (('different', 'wavelength'), 0.9106192449462366),
 (('stripe', 'size'), 0.9106192449462366),
 (('switch', 'fabric'), 0.6070794966308244),
 (('signal', 'conversion'), 0.3035397483154122),
 (('single', 'bit'), 0.3035397483154122),
 (('semisynchronous', 'time'), 0.3035397483154122),
 (('electronic', 'control'), 0.3035397483154122)]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[(('internal', 'combustion'), 0.46542761408363204),
 (('predefined', 'speed'), 0.46542761408363204),
 (('valve', 'train'), 0.46542761408363204),
 (('boundary', 'lubrication'), 0.46542761408363204),
 (('four-ball', 'tribotester'), 0.46542761408363204),
 (('acquisition', 'software'), 0.46542761408363204),
 (('Four-ball', 'tribotester'), 0.46542761408363204),
 (('top', 'ball'), 0.46542761408363204)]
'------------------------------'
'doi-cap-con/10.1155:2016:1925827.txt'
[(('lattice', 'rectangle'), 0.7480086654915

## Kategorien Mapping
8 beste Wortpaare
Nominalphrase als String ohne Sonderzeichen: ,  ) ( '  in der Funktion "def wiki_cats(term)" 
Umwandeln in Kleinbuchstaben

In [7]:
import glob
import nltk
import codecs
import pprint
import treetaggerwrapper
import math
import requests
from nltk.corpus import stopwords
import string 
import re

filelist = glob.glob("doi-cap-con/*.txt") #glob-Modul für Pfadnamen nach Muster
nr_of_docs = len(filelist) #Anzahl der Dateien
noun2cat = {} #Dictionary für Substantiv -> [Wikicats]
catStoplist = stopwords.words('categories') ####unsinnige Kategorien (z.B Numbers)
df = {} 
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
NPList = fdist2.keys()

def candidates(taglist):#Liste mit Tags, als Ausgabe als Liste
    cand = []
    skip = False
    for i in range(len(taglist)-1) :
        if skip:
            skip = False
            continue
        skip = False
        l1 = taglist[i].lemma # erstes Wort
        l2 = taglist[i+1].lemma# zweites Wort
        if len(l1) >2 and len(l2) >2:   #Die einzelnen Wörter der NP sollen eine Länge von über 2 Buchstaben haben
            if (l1,l2) in NPList:
                skip = True
                cand.append((l1,l2))
    return cand


def substantive_zaehlen(datei):
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    nouns_in_text = []   
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        for substantiv in nouns_from_sent:
            if substantiv not in nouns_in_text:
                nouns_in_text.append(substantiv)
                
    for n in nouns_in_text:
        df_n = df.get(n,0) #Der Wert von n falls n vorhanden ist, sonst 0
        df[n] = df_n + 1
    

for f in glob.glob("doi-cap-con/*.txt"):
    substantive_zaehlen(f)
    
def extract_kw(datei,nr_of_docs):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')
    
    textfile = codecs.open(datei, "r", "utf-8")
    text = textfile.read()
    textfile.close()
    
    sentences = nltk.sent_tokenize(text,language='english')
    sentences_tok = [nltk.word_tokenize(sent,language='english') for sent in sentences]
    nouns = [] 
    for sent in sentences_tok:
        tags = tagger.tag_text(sent,tagonly=True) 
        tags2 = treetaggerwrapper.make_tags(tags);
        nouns_from_sent = candidates(tags2)
        nouns.extend(nouns_from_sent)
    fdist = nltk.FreqDist(nouns)  
    
    for word in fdist:
        idf = 1.0 + math.log(float(nr_of_docs) / float(df[word]))
        fdist[word] = float(fdist[word]) / float(len(nouns)) * idf
        
    
    return fdist.most_common(8)

def wiki_cats(term):
    cats = []    
    term_string = str(term)  #Strings, damit die Zeichen ,')( entfernt werden können
    term_lower = term_string.lower() #Kleinbuchstaben, weil zweites Wort bei Wikipedia Titel klein beginnen muss
    rechar = re.compile(r"[',()]") #Ersetzen der Zeichen ,')(  zu einem Leerzeichen
    term_clear = rechar.sub("", term_lower)
     
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term_clear,  ### Bereinigte Nominalphrasen
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json()    
    
    for pageid in response['query']['pages']:
        if 'categories' in response['query']['pages'][pageid]: 
            cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14 and not cat['title'].startswith('Category:Disambiguation') and not cat['title'].startswith('Category:Wikipedia articles incorporating') and cat['title'] not in catStoplist])
        return set(cats) 

def collect_nouns(flist):
    global noun2cat
    for f in flist:
        nouns = extract_kw(f, nr_of_docs)
        for (n,f) in nouns:   
            noun2cat[n] = wiki_cats(n)   

collect_nouns(filelist[150:170])


filelist = glob.glob("doi-cap-con/*.txt")
nr_of_docs = len(filelist)
for f in filelist[150:170]:
    keywords = extract_kw(f,nr_of_docs)   
    catsPair = set([c for (kw,f) in keywords for c in noun2cat.get(kw)]) 
    pprint.pprint(f)
    pprint.pprint(catsPair)
    pprint.pprint("------------------------------")
    


'doi-cap-con/10.1155:2016:1875357.txt'
set()
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
set()
'------------------------------'
'doi-cap-con/10.1155:2016:1925827.txt'
{'Category:Artificial intelligence stubs',
 'Category:Dynamical systems',
 'Category:Models of computation',
 'Category:Stochastic processes',
 'Category:Systems theory',
 'Category:Variants of random walks'}
'------------------------------'
'doi-cap-con/10.1155:2016:1928465.txt'
set()
'------------------------------'
'doi-cap-con/10.1155:2016:1962438.txt'
{'Category:Capacitors', 'Category:Capacitance'}
'------------------------------'
'doi-cap-con/10.1155:2016:1979348.txt'
{'Category:Ligaments'}
'------------------------------'
'doi-cap-con/10.1155:2016:2136381.txt'
set()
'------------------------------'
'doi-cap-con/10.1155:2016:2175896.txt'
set()
'------------------------------'
'doi-cap-con/10.1155:2016:2303181.txt'
{'Category:Artificial neural networks',
 'Category:Computer science stubs',

### Ranking der Kategorien
Oberkategorien der Kategorien ermittel. Anzahl der Überscheidungen = Anzahl Beziehungen
Ranking nach Anzahl

In [8]:
import requests

def wiki_cats(term):
    cats = []
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': term,
            'prop': 'categories',
            'clshow':'!hidden',
            'cllimit':'50'
        }
    ).json()
    
    for pageid in  response['query']['pages']:
        cats.extend([cat['title'] for cat in response['query']['pages'][pageid]['categories'] if cat['ns'] == 14])

    return cats # Es sind doppelte Kategorien vorhanden


def intersection(a,b):
    i = 0
    for e in a:
        if e in b:
            i += 1
    return i

def wiki_cats_sort_rel(f):
    keywords = extract_kw(f,nr_of_docs) 
    cats = set([c for (kw,f) in keywords for c in noun2cat.get(kw)]) 
    extended = {}
    relnumber = {}
    for cat in cats:
        family = [cat]
        family.extend(wiki_cats(cat))
        extended[cat] = family

    for cat in cats:
        relsize = 0
        for cat2 in cats:
            if cat == cat2:
                continue
            if intersection(extended[cat],extended[cat2]) > 0:
                relsize +=1
        relnumber[cat] = relsize
    return sorted(relnumber.items(),key = lambda x:x[1],reverse=True)
            
for f in filelist[150:170]:    
    pprint.pprint(f) 
    cts = wiki_cats_sort_rel(f)
    pprint.pprint(cts)
    pprint.pprint("------------------------------") 


'doi-cap-con/10.1155:2016:1875357.txt'
[]
'------------------------------'
'doi-cap-con/10.1155:2016:1901493.txt'
[]
'------------------------------'
'doi-cap-con/10.1155:2016:1925827.txt'
[('Category:Systems theory', 1),
 ('Category:Stochastic processes', 1),
 ('Category:Variants of random walks', 1),
 ('Category:Dynamical systems', 1),
 ('Category:Models of computation', 0),
 ('Category:Artificial intelligence stubs', 0)]
'------------------------------'
'doi-cap-con/10.1155:2016:1928465.txt'
[]
'------------------------------'
'doi-cap-con/10.1155:2016:1962438.txt'
[('Category:Capacitors', 1), ('Category:Capacitance', 1)]
'------------------------------'
'doi-cap-con/10.1155:2016:1979348.txt'
[('Category:Ligaments', 0)]
'------------------------------'
'doi-cap-con/10.1155:2016:2136381.txt'
[]
'------------------------------'
'doi-cap-con/10.1155:2016:2175896.txt'
[]
'------------------------------'
'doi-cap-con/10.1155:2016:2303181.txt'
[('Category:Neuroscience stubs', 2),
 ('Categ